In [6]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [7]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://ms-mlflow-bucket/500246362129033379', creation_time=1751027568684, experiment_id='500246362129033379', last_update_time=1751027568684, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [9]:
df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

(36243, 2)

In [10]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-06-27 21:09:16,610] A new study created in memory with name: no-name-4f862267-2bfe-45da-8583-9c8bb37bef84
[I 2025-06-27 21:09:16,754] Trial 0 finished with value: 0.703404255319149 and parameters: {'alpha': 0.6933609065178119}. Best is trial 0 with value: 0.703404255319149.
[I 2025-06-27 21:09:16,786] Trial 1 finished with value: 0.7080851063829787 and parameters: {'alpha': 0.014297443489814227}. Best is trial 1 with value: 0.7080851063829787.
[I 2025-06-27 21:09:16,816] Trial 2 finished with value: 0.7072340425531914 and parameters: {'alpha': 0.04200736432184733}. Best is trial 1 with value: 0.7080851063829787.
[I 2025-06-27 21:09:16,850] Trial 3 finished with value: 0.7081914893617022 and parameters: {'alpha': 0.009665521170052363}. Best is trial 3 with value: 0.7081914893617022.
[I 2025-06-27 21:09:16,875] Trial 4 finished with value: 0.7074468085106383 and parameters: {'alpha': 0.032603374038275214}. Best is trial 3 with value: 0.7081914893617022.
[I 2025-06-27 21:09:16,900

🏃 View run MultinomialNB_SMOTE_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379/runs/56f29f607f06405dbd0d77b143450c05
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379
